In [2]:
# Install required packages if you haven't already
# !pip install lightgbm imbalanced-learn scikit-learn pandas numpy joblib

import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import train_test_split, cross_val_score, RandomizedSearchCV
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import RobustScaler
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE
import joblib
import warnings
warnings.filterwarnings('ignore')

# =============================================================================
# DATA LOADING & FEATURE ENGINEERING
# =============================================================================

# Load the data from your latest preparation script
df = pd.read_csv('processed_shark_data.csv')
print(f"Original dataset shape: {df.shape}")
print(f"Target distribution:\n{df['is_foraging'].value_counts()}")

# 1. Cyclical time features
df['hour_sin'] = np.sin(2 * np.pi * df['hour']/24.0)
df['hour_cos'] = np.cos(2 * np.pi * df['hour']/24.0)
df['month_sin'] = np.sin(2 * np.pi * df['month']/12.0)
df['month_cos'] = np.cos(2 * np.pi * df['month']/12.0)

# 2. Advanced movement and statistical features
# IMPROVEMENT: Grouping by shark 'id' for a more meaningful percentile.
df['speed_percentile'] = df.groupby('id')['speed_avg_roll'].rank(pct=True)
df['is_slow'] = (df['speed_avg_roll'] < df['speed_avg_roll'].quantile(0.25)).astype(int)
df['speed_change'] = df.groupby('id')['speed_avg_roll'].diff().fillna(0)
df['high_turn_angle'] = (df['angle_avg_roll'] > df['angle_avg_roll'].quantile(0.75)).astype(int)
df['angle_consistency'] = df['angle_std_roll'] / (df['angle_avg_roll'] + 1e-6)
df['speed_angle_ratio'] = df['speed_avg_roll'] / (df['angle_avg_roll'] + 1e-6)
df['movement_efficiency'] = df['speed_avg_roll'] / (df['speed_std_roll'] + 1e-6)
df['turning_intensity'] = df['angle_avg_roll'] * df['angle_std_roll']
df['distance_from_center'] = np.sqrt((df['lat'] - df['lat'].mean())**2 + (df['lon'] - df['lon'].mean())**2)
df['dawn_dusk'] = ((df['hour'] >= 5) & (df['hour'] <= 8) | (df['hour'] >= 17) & (df['hour'] <= 20)).astype(int)
df['night'] = ((df['hour'] >= 21) | (df['hour'] <= 4)).astype(int)
df['speed_z_score'] = (df['speed_avg_roll'] - df['speed_avg_roll'].mean()) / df['speed_avg_roll'].std()
df['angle_z_score'] = (df['angle_avg_roll'] - df['angle_avg_roll'].mean()) / df['angle_avg_roll'].std()

# Handle any NaN/Inf values created during feature engineering
df.replace([np.inf, -np.inf], np.nan, inplace=True)
df = df.fillna(df.median(numeric_only=True))

print(f"Enhanced dataset shape: {df.shape}")

# =============================================================================
# DATA PREPROCESSING
# =============================================================================

# THE FIX: Define features (X) by dropping the target and non-feature columns
X = df.drop(['is_foraging', 'id', 'date'], axis=1)
y = df['is_foraging']

# Save the column order for later
feature_columns = X.columns.tolist()
joblib.dump(feature_columns, 'feature_columns.pkl')

# Split data with stratification
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# =============================================================================
# SAMPLING & SCALING
# =============================================================================

print("\nApplying SMOTE to training set...")
# Using SMOTE as it's robust and fast
sampler = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = sampler.fit_resample(X_train, y_train)
print(f"After resample: {np.bincount(y_train_resampled)}")

# Apply robust scaling
scaler = RobustScaler()
X_train_scaled = scaler.fit_transform(X_train_resampled)
X_test_scaled = scaler.transform(X_test)

# =============================================================================
# HYPERPARAMETER OPTIMIZATION
# =============================================================================

print("\nOptimizing hyperparameters for LightGBM...")
lgbm_params = {
    'n_estimators': [200, 300, 500],
    'learning_rate': [0.01, 0.05, 0.1],
    'num_leaves': [31, 50, 100],
    'max_depth': [6, 8, 10],
    'min_child_samples': [20, 30, 50],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0],
    'reg_alpha': [0, 0.1, 0.5],
    'reg_lambda': [0, 0.1, 0.5]
}
lgbm_base = lgb.LGBMClassifier(random_state=42, verbose=-1)
lgbm_random = RandomizedSearchCV(
    lgbm_base, lgbm_params, n_iter=50, cv=5, 
    scoring='accuracy', random_state=42, n_jobs=-1
)
lgbm_random.fit(X_train_scaled, y_train_resampled)
print(f"Best LightGBM params: {lgbm_random.best_params_}")

# =============================================================================
# MODEL EVALUATION
# =============================================================================
best_lgbm = lgbm_random.best_estimator_

# Additional model for comparison
rf_model = RandomForestClassifier(n_estimators=200, max_depth=10, random_state=42, n_jobs=-1)
rf_model.fit(X_train_scaled, y_train_resampled)

# --- Create predictions ---
lgbm_pred = best_lgbm.predict(X_test_scaled)
rf_pred = rf_model.predict(X_test_scaled)
ensemble_pred_proba = (0.7 * best_lgbm.predict_proba(X_test_scaled)[:, 1]) + (0.3 * rf_model.predict_proba(X_test_scaled)[:, 1])
ensemble_pred = (ensemble_pred_proba > 0.5).astype(int)

print("\n" + "="*60)
print("MODEL EVALUATION RESULTS")
print("="*60)

# --- Select the best model based on accuracy ---
models = {
    'LightGBM': lgbm_pred,
    'Random Forest': rf_pred,
    'Ensemble': ensemble_pred
}
best_model_name = max(models.keys(), key=lambda k: accuracy_score(y_test, models[k]))
best_accuracy = accuracy_score(y_test, models[best_model_name])

print(f"\nLightGBM Accuracy: {accuracy_score(y_test, lgbm_pred):.4f}")
print(f"Random Forest Accuracy: {accuracy_score(y_test, rf_pred):.4f}")
print(f"Ensemble Accuracy: {accuracy_score(y_test, ensemble_pred):.4f}")
print(f"\nBest Model: {best_model_name}")
print(f"Best Accuracy: {best_accuracy:.4f} ({best_accuracy*100:.2f}%)")

# --- Detailed Report for the Best Model ---
print(f"\nClassification Report ({best_model_name}):")
print(classification_report(y_test, models[best_model_name], target_names=['Traveling', 'Foraging']))
print(f"\nConfusion Matrix ({best_model_name}):")
print(confusion_matrix(y_test, models[best_model_name]))

# --- Feature Importance for the Best GBM Model ---
feature_importance = pd.DataFrame({
    'feature': X.columns,
    'importance': best_lgbm.feature_importances_
}).sort_values('importance', ascending=False)
print(f"\nTop 10 Features:")
print(feature_importance.head(10))

# =============================================================================
# SAVE MODELS
# =============================================================================

# Save the best individual model and the scaler
joblib.dump(best_lgbm, 'best_shark_model_lightgbm.pkl')
joblib.dump(scaler, 'shark_model_scaler.pkl')
print("\nBest model saved as best_shark_model_lightgbm.pkl")
print("Scaler saved as shark_model_scaler.pkl")

Original dataset shape: (9149, 11)
Target distribution:
is_foraging
0    6417
1    2732
Name: count, dtype: int64
Enhanced dataset shape: (9149, 28)

Applying SMOTE to training set...
After resample: [5133 5133]

Optimizing hyperparameters for LightGBM...
Best LightGBM params: {'subsample': 0.8, 'reg_lambda': 0.5, 'reg_alpha': 0.1, 'num_leaves': 100, 'n_estimators': 500, 'min_child_samples': 20, 'max_depth': 8, 'learning_rate': 0.1, 'colsample_bytree': 0.8}

MODEL EVALUATION RESULTS

LightGBM Accuracy: 0.7634
Random Forest Accuracy: 0.7497
Ensemble Accuracy: 0.7612

Best Model: LightGBM
Best Accuracy: 0.7634 (76.34%)

Classification Report (LightGBM):
              precision    recall  f1-score   support

   Traveling       0.83      0.83      0.83      1284
    Foraging       0.61      0.60      0.60       546

    accuracy                           0.76      1830
   macro avg       0.72      0.72      0.72      1830
weighted avg       0.76      0.76      0.76      1830


Confusion Ma